<a href="https://colab.research.google.com/github/DWalicki95/MaterialsVision/blob/main/notebooks/cellpose_out_of_the_box_manual_annotations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook contains the code for evaluation of manually annotated
artificial microstructure images. This allows us to understand how high values of metrics adopted for segmentation can reach.

Code from `Model Evaluation Functions` is now implemented as a part of materials_vision repository.

In [ ]:
!git clone https://github.com/DWalicki95/MaterialsVision.git

fatal: destination path 'MaterialsVision' already exists and is not an empty directory.


In [ ]:
!pip install cellpose

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 126.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os

os.environ['PYTHONPATH'] = (
  '/content/MaterialsVision:' + os.environ.get('PYTHONPATH', '')
)

In [ ]:
from pathlib import Path
import numpy as np
from cellpose.io import imread
import tifffile
import time

from cellpose.metrics import (
    mask_ious, boundary_scores, aggregated_jaccard_index, average_precision
)
import torch
from skimage.segmentation import relabel_sequential
from cellpose.metrics import boundary_scores
from cellpose import utils
import gc
import pandas as pd
from typing import Dict, List
import json
import matplotlib.pyplot as plt

from datetime import datetime

from pycocotools.coco import COCO
import numpy as np



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.4 
platform:       	linux 
python version: 	3.11.13 
torch version:  	2.6.0+cu124! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Model evaluation functions

In [ ]:
def get_bbox_from_mask(mask: np.ndarray, relabel_image: bool = True) -> dict:
  if relabel_image:
    mask, forward, inverse_map = relabel_sequential(mask, offset=1)
  labels = np.unique(mask)
  bboxes = {}
  for label in labels[labels>0]:
    rows, cols = np.where(mask==label)
    if rows.size == 0:
      continue
    y_min, y_max = rows.min(), rows.max()
    x_min, x_max = cols.min(), cols.max()
    bboxes[int(label)] = int(x_min), int(y_min), int(x_max), int(y_max)
  return bboxes

In [ ]:
def calculate_iou(bbox_true, bbox_pred, epsilon=1e-5):
  # coordinates of the intersection box
  x1 = np.max([bbox_true[0], bbox_pred[0]])
  y1 = np.max([bbox_true[1], bbox_pred[1]])
  x2 = np.min([bbox_true[2], bbox_pred[2]])
  y2 = np.min([bbox_true[3], bbox_pred[3]])

  # area of overlap
  width = (x2 - x1)
  height = (y2 - y1)

  # if there is no overlap
  if (width<0) or (height<0):
    return 0.0
  area_of_overlap = width * height
  # combined area
  area_bbox_true = (bbox_true[2] - bbox_true[0]) * (bbox_true[3] - bbox_true[1])
  area_bbox_pred = (bbox_pred[2] - bbox_pred[0]) * (bbox_pred[3] - bbox_pred[1])
  area_combined = area_bbox_true + area_bbox_pred - area_of_overlap
  # ratio of area of overlap over combined area
  iou = area_of_overlap / (area_combined + epsilon)
  return iou

In [ ]:
def get_iou_matrix(bboxes_true, bboxes_pred):
  n_bboxes_true = len(bboxes_true)
  n_bboxes_pred = len(bboxes_pred)
  I = np.zeros((n_bboxes_true, n_bboxes_pred), dtype=np.float32)
  for i in range(n_bboxes_true):
    for j in range(n_bboxes_pred):
      I[i, j] = calculate_iou(bboxes_true[i], bboxes_pred[j])
  return I

In [ ]:
def calculate_pores_difference(n_pores_true: int, n_pores_pred):
  return float(np.abs(n_pores_true - n_pores_pred) / n_pores_true)

In [ ]:
def greedy_match_iou(iou_matrix, threshold):
  I = iou_matrix.copy()
  all_triples_list = []
  matched_true, matched_pred = set(), set()
  matches = []
  I[I < threshold] = 0.0
  N_true, N_pred = I.shape
  # get pores difference
  pores_diff = calculate_pores_difference(N_true, N_pred)

  non_zero = np.argwhere(I>0.0)
  for (i, j) in non_zero:
    all_triples_list.append((i, j, I[i, j]))

  sorted_all_triples_list = sorted(
      all_triples_list, key=lambda x: x[2], reverse=True
  )

  for triples in sorted_all_triples_list:
    if (triples[0] not in matched_true) & (triples[1] not in matched_pred):
      matched_true.add(triples[0])
      matched_pred.add(triples[1])
      matches.append((triples[0], triples[1], triples[2]))

  TP = len(matches)
  FN = N_true - len(matched_true)
  FP = N_pred - len(matched_pred)

  precision = (TP / (TP + FP) if (TP + FP) > 0 else 0.0)
  recall = (TP / (TP + FN) if (TP+FN) > 0 else 0.0)
  f1_score = (2 * (precision * recall) / (precision + recall) if
      (precision + recall) > 0 else 0.0
  )
  iou_mean = float(sum(score for (_, _, score) in matches) / TP if TP > 0 else 0.0)

  return {
      'TP': TP,
      'FP': FP,
      'FN': FN,
      'precision': precision,
      'recall': recall,
      'f1_score': f1_score,
      'iou_mean': iou_mean,
      'pores_diff': pores_diff
  }


In [ ]:
def iou_scores_batch(true_masks, pred_masks, greedy_match_threshold = 0.5):
  iou_results = {}
  for idx, (true_mask, pred_mask) in enumerate(zip(true_masks, pred_masks)):
    pred_bboxes = get_bbox_from_mask(pred_mask)
    true_bboxes = get_bbox_from_mask(true_mask)
    iou_matrix = get_iou_matrix(
        list(pred_bboxes.values()), list(true_bboxes.values())
    )
    one_img_output = greedy_match_iou(iou_matrix, greedy_match_threshold)
    iou_results[idx] = one_img_output
  return iou_results

In [ ]:
def boundary_scores_batched(masks_true, masks_pred, scales, batch_size=2):
    N = len(masks_true)
    M = len(scales)

    precision_all = np.zeros((M, N), dtype=float)
    recall_all    = np.zeros((M, N), dtype=float)
    fscore_all    = np.zeros((M, N), dtype=float)

    for start in range(0, N, batch_size):
        end = min(start + batch_size, N)
        # part of lists (batched solution)
        sub_true = masks_true[start:end]
        sub_pred = masks_pred[start:end]
        p_sub, r_sub, f_sub = boundary_scores(sub_true, sub_pred, scales)

        precision_all[:, start:end] = p_sub
        recall_all[:, start:end]    = r_sub
        fscore_all[:, start:end]    = f_sub

        # clean memory
        gc.collect()

    return precision_all, recall_all, fscore_all


In [ ]:
def summarize_evaluation_iou(iou_results):
  TP_all, FP_all, FN_all = [], [], []
  micro_metrics = ['TP', 'FP', 'FN']
  macro_metrics_list = [
      metric for metric in list(
          [metric for metric in iou_results.values()][0].keys()
      ) if metric not in micro_metrics
  ]
  macro_metric_results = {}
  # macro evaluation
  for metric_dict in iou_results.values():
    for metric, value in metric_dict.items():
      if metric not in ['TP', 'FP', 'FN']:
        macro_metric_results[f'iou_mean_{metric}'] = np.mean(value)
  # micro evaluation
      elif metric == 'TP':
        TP_all.append(value)
      elif metric == 'FP':
        FP_all.append(value)
      elif metric == 'FN':
        FN_all.append(value)
  TP_all_sum = np.sum(TP_all)
  FP_all_sum = np.sum(FP_all)
  FN_all_sum = np.sum(FN_all)
  precision_micro = TP_all_sum / (TP_all_sum + FP_all_sum)
  recall_micro = TP_all_sum / (TP_all_sum + FN_all_sum)
  f1_micro = (
      2 * precision_micro * recall_micro
    ) / (precision_micro + recall_micro)
  # prepare final report
  report = pd.DataFrame.from_dict(
      macro_metric_results, orient='index', columns=['value']
    )
  report.loc['iou_precision_micro'] = precision_micro
  report.loc['iou_recall_micro'] = recall_micro
  report.loc['iou_f1_micro'] = f1_micro
  return report

In [ ]:
def summarize_evaluation_boundary_score(precision_all, recall_all, fscore_all):
  bs_precision_mean = np.mean(precision_all)
  bs_recall_mean = np.mean(recall_all)
  bs_fscore_mean = np.mean(fscore_all)
  bs_metrics_dict = {
        'boundary_score_mean_precision': bs_precision_mean,
        'boundary_score_mean_recall':    bs_recall_mean,
        'boundary_score_mean_f1':        bs_fscore_mean,
  }
  report = pd.DataFrame.from_dict(
      bs_metrics_dict, orient='index', columns=['value']
  )
  return report

In [ ]:
def get_today_datetime_str():
  today_datetime = datetime.now().strftime("%m/%d/%Y %H:%M:%S")
  today_datetime = today_datetime[:-3]  # skip seconds
  today_datetime = (
      today_datetime
      .replace('-', '_')
      .replace(':', '_')
      .replace('/', '_')
      .replace(' ', '__')
  )
  return today_datetime

In [ ]:
def summarize_evaluation(
    output_filename: str,
    iou_results: Dict,
    boundary_score_precision_all: List,
    boundary_score_recall_all: List,
    boundary_score_fscore_all: List,
    evaluation_comment: str = '',
    save_raw_iou_results: bool = True,
    save_final_results: bool = True,
    output_directory_path = None,
    add_today_datetime_to_filename: bool = True
):
  report_iou = summarize_evaluation_iou(iou_results)
  report_boundary_score = summarize_evaluation_boundary_score(
      precision_all=boundary_score_precision_all,
      recall_all=boundary_score_recall_all,
      fscore_all=boundary_score_fscore_all
  )
  report_final = pd.concat([report_iou, report_boundary_score])
  report_final['comment'] = evaluation_comment
  # save files
  today_datetime_str = (
      get_today_datetime_str() if add_today_datetime_to_filename else ''
  )
  if not output_directory_path:
    output_directory_path = Path(
        '/content/drive/MyDrive/evaluation_metrics'
    )
  # save raw iou
  if save_raw_iou_results:
    output_directory_path.mkdir(parents=True, exist_ok=True)
    output_filename = (
        f'raw_iou_results_{evaluation_comment}_{today_datetime_str}.json'
    )
    output_filepath = output_directory_path / output_filename
    with open(output_filepath, 'w') as f:
      json.dump(iou_results, f)
    print('Raw iou test dictionary saved.')
  # save final results
  if save_final_results:
    output_directory_path.mkdir(parents=True, exist_ok=True)
    output_filename = (
        f'evaluation_results_{today_datetime_str}.csv'
    )
    output_filepath = output_directory_path / output_filename
    report_final.to_csv(output_filepath)
    print('Final evaluation metrics report saved.')
    return report_final

In [ ]:
def visualize_results(
    pred_masks,
    img_idx_to_visualize: int = 0,
    save_subsample: bool = True,
    output_directory_path = None,
    add_today_datetime_to_filename: bool = True,
    evaluation_comment: str = ''
):
  if not output_directory_path:
    output_directory_path = Path(
        '/content/drive/MyDrive/evaluation_metrics/plots'
    )
  today_datetime_str = (
      get_today_datetime_str() if add_today_datetime_to_filename else ''
  )
  # save images
  if save_subsample:
    output_directory_path.mkdir(parents=True, exist_ok=True)
    num_images = 10
    for img_idx in range(num_images):
        fig = plt.figure(figsize=(20, 12))
        plot.show_segmentation(
          fig=fig,
          maski=pred_masks[img_idx],
          img=imgs[img_idx],
          flowi=flows[img_idx][0]
        )
        plt.savefig(
            f'{output_directory_path}/{evaluation_comment}_figure_{img_idx}.jpg'
        )
        plt.close(fig)
  # show_figure
  fig = plt.figure(figsize=(20, 12))
  plot.show_segmentation(
    fig=fig,
    maski=pred_masks[img_idx_to_visualize],
    img=imgs[img_idx_to_visualize],
    flowi=flows[img_idx_to_visualize][0]
  )
  plt.show()

# Load manual annotations

In [ ]:
def load_manual_annotated_image(
    search_img_name: str,
    folder_path=r'/content/manual_annotated_images'
  ) -> np.ndarray:
    '''
    Loads images. Remember to firstly upload files to
    folder_path
    '''
    matched = list(Path(folder_path).glob(search_img_name))
    return imread(matched[0])

In [ ]:
def load_manual_annotations(result_filepath: Path) -> pd.DataFrame:
  '''
  Creates result dataframe with images, their predicted and true
  masks.
  '''
  result_df = pd.DataFrame(columns=[
      'img_id', 'img_name', 'img', 'pred_mask', 'true_mask'
    ]
  )

  coco = COCO(filepath_json)
  imgs_ids = coco.getImgIds()
  imgs_info = coco.loadImgs(imgs_ids)
  for img_id, img_info in zip(imgs_ids, imgs_info):
    h, w = img_info["height"], img_info["width"]
    img_name = img_info['file_name'].split('/')[-1].split('-')[-1]

    anns = coco.loadAnns(coco.getAnnIds(imgIds=[img_id]))
    pred_mask = np.zeros((h, w), dtype=np.uint16)

    for idx, ann in enumerate(anns, start=1):
        m = coco.annToMask(ann)   # mask 0/1
        pred_mask[m > 0] = idx

    # read proper image
    img = load_manual_annotated_image(img_name)
    # read true masks
    true_mask_filename = '_'.join(img_name.split('_')[:2]) + '_masks.tiff'
    true_mask = load_manual_annotated_image(
        search_img_name=true_mask_filename,
        folder_path=r'/content/true_masks'
    )
    # add results
    result_df.loc[len(result_df)] = [
        img_id, img_name, img, pred_mask, true_mask
      ]

  return result_df

In [ ]:
filepath_json = Path(r'/content/result.json')  # add result.json file extracted
# as COCO file using Label Studio
df_manual_annotations = load_manual_annotations(filepath_json)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


# Evaluation

In [ ]:
true_masks = list(df_manual_annotations['true_mask'].values)
pred_masks = list(df_manual_annotations['pred_mask'].values)

In [ ]:
iou_results = iou_scores_batch(
    true_masks=true_masks,
    pred_masks=pred_masks,
    greedy_match_threshold=0.5
)

In [ ]:
bs_precision_all, bs_recall_all, bs_fscore_all = boundary_scores_batched(
    true_masks,
    pred_masks,
    scales=[0.1],  # scale can be changed
    batch_size=3
)

In [ ]:
summarize_evaluation(
    output_filename='eveluation_mannual_annotations',
    iou_results=iou_results,
    boundary_score_precision_all=bs_precision_all,
    boundary_score_recall_all=bs_recall_all,
    boundary_score_fscore_all=bs_fscore_all,
    evaluation_comment='eveluation_mannual_annotations',
    save_raw_iou_results=True,
    save_final_results=True,
    add_today_datetime_to_filename=True
)

Raw iou test dictionary saved.
Final evaluation metrics report saved.


,value,comment
iou_mean_precision,1.000000,eveluation_mannual_annotations
iou_mean_recall,1.000000,eveluation_mannual_annotations
iou_mean_f1_score,1.000000,eveluation_mannual_annotations
iou_mean_iou_mean,0.939868,eveluation_mannual_annotations
iou_mean_pores_diff,0.000000,eveluation_mannual_annotations
iou_precision_micro,1.000000,eveluation_mannual_annotations
iou_recall_micro,1.000000,eveluation_mannual_annotations
iou_f1_micro,1.000000,eveluation_mannual_annotations
boundary_score_mean_precision,0.923976,eveluation_mannual_annotations
boundary_score_mean_recall,0.995515,eveluation_mannual_annotations
